# Análise Exploratória dos Dados - Competição "Talking Data"
## Alunos: Alessandro Rivello e Raul Guarini

Fazemos uma análise dos dados de teste e treinamento neste notebook. No caso do dados de treinamento, trabalhamos com uma amostra de 10% dos dados originais, frente ao tamanho do arquivo train.csv (quase 190 milhões de linhas!!!!!). O algoritmo para escolha da amostra está disponível no script ```row_select.py```.

Ambos datasets continham a coluna "click_time" com o timestamp do acesso no formato "YYYY-MM-DD hh:mm:ss". Para poder trabalhar os dados, utilizamos o comando ```sed``` no Terminal para:
1. Eliminar informação sobre o ano e o mês, em vista do fato de que todos os dados se encontram numa janela de apenas quatro dias. Isto torna o dataset mais leve e mais fácil de ser manipulado;
2. Colocar as colunas de dias, horas, minutos e segundos separadamente, facilitando também a análise.

No caso dos dados de teste, por exemplo, o comando utilizado foi: 
```
cat test.csv | sed 's/2017-11-//g' | sed -E -e 's/[[:space:]]/,/g' -e 's/:/,/g' -e 's/click_time/days,hours,minutes,seconds/g' > test_timed.csv

```

Também julgamos desnecessária a coluna "attributed_time". É um timestamp do horário em que o download foi feito. Como veremos, ela aporta pouca informação pois o dataset é amplamente desbalanceado, com poucos downloads efetivos. Além disso, este dado é gerado após o download ser feito, o que o torna incompatível com a tarefa de prever se o download ocorrerá ou não. Portanto, os dados utilizados estão disponíveis nos arquivos ```10_train_timed.csv``` e ```test_timed.csv```.

Links do Dropbox para estes arquivos (zipados):
1. ```10_train_timed.csv```: https://www.dropbox.com/s/d16l70ib5y0tjyr/10_train_timed.csv.zip?dl=0
2. ```test_timed.csv```: https://www.dropbox.com/s/gg14d21zskkqg7r/test_timed.csv.zip?dl=0

### Importando os dados

In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import datetime

# Definindo o estilo dos plots
sns.set_style("whitegrid")
sns.set(font_scale=1.5)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Mudar o path conforme necessário
path_raul = '/Users/Raul/Desktop/'